In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
import xgboost as xgb
import math
from sklearn.model_selection import RandomizedSearchCV
import datetime as dt
import pytz
import holidays
from geopy.distance import geodesic

In [58]:
df = pd.read_parquet('Resources/model_df.parquet')

df.columns = df.columns.str.replace("'", "")
df.columns = df.columns.str.replace("(", "")
df.columns = df.columns.str.replace(")", "")
df.columns = df.columns.str.replace(",", "")

df = df.drop(['fare', 'PUx', 'PUy', 'DOx', 'DOy', 'durationsec'], axis=1)

df

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,distance,tolls,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,False,1,0,0,0.0,42.80,18.85,6.94,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,2.47,0.00,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,2.02,0.00,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,2.47,0.00,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,1.06,0.00,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,2.48,0.00,...,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,1.58,0.00,...,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,0.60,0.00,...,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,1.36,0.00,...,False,False,False,True,False,False,True,False,False,False


In [59]:
df.columns

Index(['second_of_day', 'day_of_year', 'weekend', 'holiday', 'morning_rush',
       'evening rush', 'prcp', 'temp', 'distance', 'tolls', 'airport',
       'congestion', 'total', 'class', 'PU_Bronx', 'PU_Brooklyn',
       'PU_Manhattan', 'PU_Queens', 'PU_Staten Island', 'DO_Bronx',
       'DO_Brooklyn', 'DO_Manhattan', 'DO_Queens', 'DO_Staten Island'],
      dtype='object')

In [60]:
df_2 = pd.read_parquet('Resources/model_df.parquet')

df_2.columns = df_2.columns.str.replace("'", "")
df_2.columns = df_2.columns.str.replace("(", "")
df_2.columns = df_2.columns.str.replace(")", "")
df_2.columns = df_2.columns.str.replace(",", "")

df_2 = df_2.drop(['fare', 'distance'],axis=1)

df_2

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,...,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,...,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,...,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,...,False,False,False,True,False,False,True,False,False,False


In [61]:
df_2.columns

Index(['second_of_day', 'day_of_year', 'weekend', 'holiday', 'morning_rush',
       'evening rush', 'prcp', 'temp', 'PUx', 'PUy', 'DOx', 'DOy',
       'durationsec', 'tolls', 'airport', 'congestion', 'total', 'class',
       'PU_Bronx', 'PU_Brooklyn', 'PU_Manhattan', 'PU_Queens',
       'PU_Staten Island', 'DO_Bronx', 'DO_Brooklyn', 'DO_Manhattan',
       'DO_Queens', 'DO_Staten Island'],
      dtype='object')

In [62]:
X = df.copy()
X = X.drop('total', axis=1)

y = df['total'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29)


In [63]:
X_2 = df_2.copy()
X_2 = X_2.drop('durationsec', axis=1)

y_2 = df_2['durationsec'].values.reshape(-1,1)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y_2, random_state=29)


In [64]:
xgbr = xgb.XGBRegressor(objective='reg:squarederror',
                        subsample= 0.8,
                        n_estimators= 300,
                        max_depth = 9,
                        learning_rate = 0.1,
                        gamma= 0.1,
                        colsample_bytree= 0.6,
                        seed=29)

In [65]:
xgbr2 = xgb.XGBRegressor(objective='reg:squarederror',
                        subsample= 0.8,
                        n_estimators= 300,
                        max_depth = 9,
                        learning_rate = 0.1,
                        gamma= 0.1,
                        colsample_bytree= 0.6,
                        seed=29)

In [66]:
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [67]:
#xgbr2.fit(X_train2, y_train2)

In [68]:
xgbr.score(X_train, y_train)

0.8835031141539589

In [69]:
xgbr.score(X_test, y_test)

0.8739617304404815

In [70]:
#xgbr2.score(X_train2, y_train2)

In [71]:
#xgbr2.score(X_test2, y_test2)

In [72]:
y_pred = xgbr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
print(mse)
print(rmse)

52.88837780674944
7.272439604888406


In [73]:
#y_pred_2 = xgbr2.predict(X_test2)
#mse2 = mean_squared_error(y_test2, y_pred_2)
#rmse2 = math.sqrt(mse2)
#print(mse2)
#print(rmse2)

In [74]:
mean_absolute_error(y_test, y_pred)

3.9930032010458727

In [75]:
#mean_absolute_error(y_test2, y_pred_2)

In [76]:
df.columns

Index(['second_of_day', 'day_of_year', 'weekend', 'holiday', 'morning_rush',
       'evening rush', 'prcp', 'temp', 'distance', 'tolls', 'airport',
       'congestion', 'total', 'class', 'PU_Bronx', 'PU_Brooklyn',
       'PU_Manhattan', 'PU_Queens', 'PU_Staten Island', 'DO_Bronx',
       'DO_Brooklyn', 'DO_Manhattan', 'DO_Queens', 'DO_Staten Island'],
      dtype='object')

In [77]:
df['congestion'].value_counts()

congestion
0.00    2621998
2.50    1806609
2.75    1578171
0.75      35850
5.50       1819
Name: count, dtype: int64

In [78]:
features = [
            'second_of_day',
            'day_of_year',
            'weekend',
            'holiday',
            'morning_rush',
            'evening rush',
            'prcp',
            'temp',
            'distance',
            'tolls',
            'airport',
            'congestion',
            'class',
            'PU_Bronx',
            'PU_Brooklyn',
            'PU_Manhattan',
            'PU_Queens',
            'PU_Staten Island',
            'DO_Bronx',
            'DO_Brooklyn',
            'DO_Manhattan',
            'DO_Queens',
            'DO_Staten Island']


In [ ]:
timezone = pytz.timezone('America/New_York')
now = dt.datetime.now(timezone)
us_holidays = holidays.US(state='NY')

coords = {
    'Bronx': (40.8370, -73.8654),
    'Brooklyn': (40.6782, -73.9442),
    'Manhattan': (40.7128, -74.0060),
    'Queens': (40.7282, -73.7949),
    'Staten Island': (40.5790, -74.1515)
}

test_input = {
            'second_of_day': [(now.hour * 3600) + (now.minute * 60) + now.second],
            'day_of_year': [now.timetuple().tm_yday],
            'weekend': [now.strftime('%A') in ['Saturday', 'Sunday']],
            'holiday': [0], # do not change
            'morning_rush': [0], # do not change
            'evening rush': [0], # do not change
            'prcp': [0],
            'temp': [43],
            'distance': [0], # do not change
            'tolls': [0],
            'airport': [0],
            'congestion': [2.50],
            'class': [0], # do not change
            'PU_Bronx': [1],
            'PU_Brooklyn': [0],
            'PU_Manhattan': [0],
            'PU_Queens': [0],
            'PU_Staten Island': [0],
            'DO_Bronx': [0],
            'DO_Brooklyn': [1],
            'DO_Manhattan': [0],
            'DO_Queens': [0],
            'DO_Staten Island': [0]}

test_input['holiday'] = [1 if now.date() in us_holidays else 0]

day_of_week = now.strftime('%A')
hour = now.hour
test_input["morning_rush_hour"] = [1 if day_of_week in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] and 7 <= hour <= 9 else 0]

test_input["evening_rush_hour"] = [1 if day_of_week in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] and 16 <= hour <= 18 else 0]


def calc_distance(coord1, coord2):
    return geodesic(coord1, coord2).miles

pickup = None
dropoff = None

for key, value in test_input.items():
    if key.startswith('PU_') and test_input[key][0] == 1:
        pickup = key[3:]
    elif key.startswith('DO_') and test_input[key][0] == 1:
        dropoff = key[3:]


print(f"Pickup borough: {pickup}")
print(f"Dropoff borough: {dropoff}")

calculated_distance = None
if pickup and dropoff and pickup in coords and dropoff in coords:
    pu_coords = coords[pickup]
    do_coords = coords[dropoff]
    calculated_distance = round(calc_distance(pu_coords, do_coords), 2)
else:
    print("Could not determine both pickup and dropoff boroughs or their coordinates.")

test_input['distance'] = [calculated_distance]

test_input


Pickup borough: Bronx
Dropoff borough: Brooklyn


{'second_of_day': [42501],
 'day_of_year': [75],
 'weekend': [True],
 'holiday': [0],
 'morning_rush': [0],
 'evening rush': [0],
 'prcp': [2],
 'temp': [43],
 'distance': [11.71],
 'tolls': [0],
 'airport': [0],
 'congestion': [2.5],
 'class': [0],
 'PU_Bronx': [1],
 'PU_Brooklyn': [0],
 'PU_Manhattan': [0],
 'PU_Queens': [0],
 'PU_Staten Island': [0],
 'DO_Bronx': [0],
 'DO_Brooklyn': [1],
 'DO_Manhattan': [0],
 'DO_Queens': [0],
 'DO_Staten Island': [0],
 'morning_rush_hour': [0],
 'evening_rush_hour': [0]}

In [100]:
calculated_distance

11.71

In [101]:
test_df = pd.DataFrame(test_input)
test_df = test_df[features]
results_pred = xgbr.predict(test_df)
print(f'Total predicted Cab Price: ${results_pred[0]:.2f}')

test_df_2 = test_df.copy()
test_df_2['class'] = 1

results_pred_2 = xgbr.predict(test_df_2)
print(f'Total predicted Uber Price: ${results_pred_2[0]:.2f}')

test_df_3 = test_df.copy()
test_df_3['class'] = 2
results_pred_3 = xgbr.predict(test_df_3)
print(f'Total predicted Lyft Price: ${results_pred_3[0]:.2f}')

Total predicted Cab Price: $62.88
Total predicted Uber Price: $49.58
Total predicted Lyft Price: $51.81


In [107]:
lowest_results = {'Yellow Cab': results_pred[0], 'Uber': results_pred_2[0], 'Lyft': results_pred_3[0]}
lowest_results_df = pd.DataFrame(lowest_results, index=[0])
min_value = lowest_results_df.loc[0].min()
min_service = lowest_results_df.loc[0].idxmin()
max_value = lowest_results_df.loc[0].max()
max_service = lowest_results_df.loc[0].idxmax()

print(f'For Pickup in {pickup} and Dropoff in {dropoff},\nWith an avg distance of {calculated_distance} miles,\n{min_service} has the lowest predicted total price of ${min_value:.2f}.\nWith {max_service} having the highest predicted total price of ${max_value:.2f}.')


For Pickup in Bronx and Dropoff in Brooklyn,
With an avg distance of 11.71 miles,
Uber has the lowest predicted total price of $49.58.
With Yellow Cab having the highest predicted total price of $62.88.
